# Data Validation and Checks (Notebook 1)

The first step is always loading the data and make yourself familiar with its size, identifiers, features etc.

Things to look out for:
 * What is the **size of the dataset**? Is down-sampling needed?
 * **Which columns are present**? And which seem most important? Is there a structure?
 * Are there **outliers** and / or **inplausible values**?
 * Is **data missing**? If so, how many fields are missing? In which columns / rows and what might be reasons?
 

# Loading the Data

We are using the [CO2 dataset provided by our world in data](https://ourworldindata.org/co2-dataset-sources).

In [ ]:
import pandas as pd
import numpy as np
import datetime

You should find the data already checked into the data folder. Otherwise, you can download them by uncommenting:

In [ ]:
#!wget https://github.com/owid/co2-data/raw/master/owid-co2-data.csv -O ../data/owid-co2-data.csv
raw_data = pd.read_csv("../data/owid-co2-data.csv")

#!wget https://github.com/owid/co2-data/raw/master/owid-co2-codebook.csv -O ../data/owid-co2-codebook.csv
raw_data_info = pd.read_csv("../data/owid-co2-codebook.csv").set_index("column").description
raw_data_source = pd.read_csv("../data/owid-co2-codebook.csv").set_index("column").source

Some plotting libraries we are going to use:

In [ ]:
import plotly.express as px
import altair as alt
from IPython.display import display

# First Impression of our dataset

* How many data do we have?
    * Rows
    * Columns
* Which data types are contained

In [ ]:
raw_data

In [ ]:
raw_data.dtypes.value_counts().plot.barh()

As there are over 50 float columns, their description is quite verbose

In [ ]:
raw_data.describe()

More interesting are the Non-Float Columns such as country

In [ ]:
raw_data.country.value_counts().hist(bins=50, figsize=(20,5))

The plotting (such as `hist`, `plot`...) commands of pandas can also be used with other plotting libraries using the `backend` parameter

In [ ]:
raw_data.country.value_counts().hist(backend="plotly")

# Libraries for EDA 

Of course, the python eco-system also provides multiple libraries to support you with the explorative analysis of data. To mention a few:

* [Dataprep](https://dataprep.ai/)
* [SweetViz](https://github.com/fbdesignpro/sweetviz)
* [PandasProfiling](https://github.com/ydataai/pandas-profiling)
* [Dtale](https://github.com/man-group/dtale)

For more info you might read: [Comparing the Five Most Popular EDA Tools](https://towardsdatascience.com/comparing-five-most-popular-eda-tools-dccdef05aa4c)

In [ ]:
from dataprep.eda import plot
from dataprep.eda import create_report
report = create_report(raw_data)
report.save('Dataprep Report on Countries')
report.show_browser()

In [ ]:
plot(raw_data, "year")

# Analyzing Columns

EDA tooling offers a great way to easily get an overview of certain columns. How ever, for datasets such as this, there is way to much information to read over all of them. Therefore it is a good idea to have a closer look at the column names to discover some structure

In [ ]:
raw_data.columns

In [ ]:
search_terms = ["capita", "relative", "cumulative", "per_gdp", "share"]

df = pd.DataFrame([[c, *[term in c for term in search_terms]] for c in raw_data.columns], columns=["column"] + search_terms)

In [ ]:
df.groupby(search_terms).count()

Lets use this information to create some groups of column names to filter them easily

In [ ]:
def cols(filter_term = None):
    if filter_term is None:
        return raw_data.columns
    else:
        return [c for c in raw_data.columns if filter_term in c]
        
col_keys = ["country", "year"]
cols_per_capita = cols("capita")
cols_relative = cols("relative")
cols_cumulative = cols("cumulative")
cols_per_gdp = cols("per_gdp")
cols_share = cols("share")

In [ ]:
cols_not_calculated = raw_data_info.drop(cols_per_capita + cols_relative + cols_cumulative + cols_share + cols_per_gdp).index.tolist()[1:]

In [ ]:
cols_not_calculated

In [ ]:
cols_co2_sources = ["cement_co2", "coal_co2", "flaring_co2", "gas_co2", "oil_co2", "other_industry_co2"]

# Outliers and implausible values

Another important task in the beginning is to look out for unexpected dataset. For example in this dataset, we expect the rows to be countries. How ever, we can quickly see that there are also aggregations of single countries contained:

    * by contintent
    * by income-groups
    * Some other groups
    
As we don't want to count cases multiple times, we will have to divide those groups

In [ ]:
countries = raw_data[~raw_data.iso_code.fillna("OWID").str.contains("OWID")]
not_countries = raw_data[raw_data.iso_code.fillna("OWID").str.contains("OWID")]

In [ ]:
countries.country.value_counts()

In [ ]:
not_countries.fillna("").groupby(["iso_code", "country"]).size()

In [ ]:
world = raw_data.query("country=='World'")
continents = raw_data[raw_data.country.isin(["Europe", "North America", "Asia", "Africa", "South America", "Oceania"])]
incomes = raw_data[raw_data.country.str.contains("-income countries")]

# Analyzing Missing Values

percentage of cells with missing data

In [ ]:
raw_data.isna().mean().mean()

## Which columns have little missing values?

In [ ]:
columns_most_present = (1- raw_data[cols_not_calculated].isna().mean()).sort_values()

In [ ]:
columns_most_present.plot.barh(figsize=(10,10), title="Fraction of columns with data")

## 📝 Which countries offer the best data?

Create a similiar chart as above, to compare the amount of missing data per country

In [ ]:
plot_df = raw_data.groupby("country").apply(lambda x: 1-x.isna().mean().mean()).sort_values()
plot_df.plot.barh(title="data_information_by_country", figsize=(20,50))

## How does time affect missing values?

In [ ]:
raw_data.groupby("year").apply(lambda x: x.isna().mean().mean()).plot.line(figsize=(15, 5))

## How do countries and columns relate?

In [ ]:
percentage_na_per_country = raw_data.groupby("country").apply(
    lambda df: 1-df.select_dtypes(np.number).isna().mean()
).unstack().reset_index().rename(columns={"level_0": "variable", 0: "columns"})

In [ ]:
heatmap = countries.groupby("country").apply(
    lambda df: df.select_dtypes(np.number).isna().mean()
).reset_index()

In [ ]:
heatmap.style.format(precision=2).background_gradient(cmap='coolwarm')

As this table is really long, maybe it is better to only include some countries and columns:

In [ ]:
filter_most_recent = lambda x: x.year == 2020
top_20_most_emitting_countries = countries[filter_most_recent].set_index("country").sort_values("co2", ascending=False).head(20)

In [ ]:
table_style = [dict(selector="th", props=[('max-width', '80px'),
                                ('text-overflow', 'ellipsis'), ('overflow', 'hidden')])]

heatmap.loc[
    heatmap.country.isin(top_20_most_emitting_countries.index), cols_not_calculated
].style.format(precision=2).background_gradient(cmap='coolwarm').set_table_styles(table_style)

Of course, such a heatmap could also be created with some visualization tools, here Altair. Don't mind the complicated code, we will introduce Altair later:

In [ ]:
plot_df = percentage_na_per_country.loc[
    (percentage_na_per_country.country.isin(top_20_most_emitting_countries.index)) &
    (percentage_na_per_country.variable.isin(cols_not_calculated))
]

alt.data_transformers.disable_max_rows()
alt.Chart(
    plot_df, 
    title="Fraction of data with data"
).mark_rect().encode(
    x=alt.X('variable',  sort=alt.SortArray(list(cols_not_calculated))),
    y=alt.Y('country', sort=alt.SortArray(list(top_20_most_emitting_countries.index))),
    color=alt.Color('columns', scale=alt.Scale(scheme='redyellowgreen')),
    tooltip=["country","variable", "columns"]
)

# Results

* The rows of the dataset contain **countries** and **country aggregations**
* The dataset contains over 50 columns with mostly numerical values, but many of those are somehow calculated (relationships, aggregations...)
* The key columns are:
    * country
    * year
    
* We have over 50% missing data. The reason for this are:
    * Some countries have a longer tracking history then others
    * More general metrics (co2, population) are tracked longer then others (co2 sources, co2 trading)
    
* We can re-use some of our datastructure (column-groups, splitted datasets, filters)